npy -> {import numpy as np
       {np.load
       
npz -> {import scipy.sparse as sp
       {sp.load_npz

Remark : problem in load_data.
         add : n_test = X_test.shape[0]
         modify in X.test hstack : replace n by n_test

TP 07/01/2022 : SD-TSIA 211 - Optimization for machine learning - NLP

In [9]:
import numpy as np
import scipy.sparse as sp
from scipy.linalg import norm, solve
from scipy.optimize import check_grad
import matplotlib.pyplot as plt
import time

def load_data(file_name_matrix='tfidf_matrix_97MB.npz', file_name_feature_names='feature_names_97MB.npy',
	      file_name_labels='train_labels.npy', samples_in_train_set=10000,
	      samples_in_test_set=137562):
	# Recuperation des donnees
	TF_IDF_matrix = sp.load_npz(file_name_matrix)
	TF_IDF_feature_names = np.load(file_name_feature_names)
	train_labels = np.load(file_name_labels, allow_pickle=True)
	train_labels_numeric = ((train_labels == 'Oui') + 0)

	X = TF_IDF_matrix[:samples_in_train_set].toarray()
	y = train_labels_numeric[:samples_in_train_set] * 2 - 1

	X_test = TF_IDF_matrix[samples_in_train_set:samples_in_train_set+samples_in_test_set].toarray()
	y_test = train_labels_numeric[samples_in_train_set:samples_in_train_set+samples_in_test_set] * 2 - 1


	# Standardisation des données
	std_X = np.maximum(np.std(X, axis=0), 1e-7)
	X = X / std_X
	X_test = X_test / std_X

	n = X.shape[0]
	n_test = X_test.shape[0]
	m = X.shape[1]

	# Ajout d'une colonne de uns
	eX = np.hstack((np.ones((n,1)), X))
	eX_test = np.hstack((np.ones((n_test,1)), X_test))

	return eX, y, eX_test, y_test

eX, y, eX_test, y_test = load_data()
y

array([-1, -1,  1, ..., -1,  1,  1])

**<h4>Q3.1/ Gradient and Hessian computations </h4>**
**The function :** $$f_{1}: (\omega_0,\omega) \mapsto \frac{1}{n} \sum\limits_{i=1}^n \log (1 +  \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0} ))) + \frac{\rho}{2} \left\|w\right\|^{2}_{2}$$

**The gradient, a vector of size p+1 (p from $\omega$, 1 from $\omega_{0}$) :**

$$\nabla f_{1}: (\omega_0,\omega) \mapsto \left[\begin{array}{c}
\frac{1}{n} \sum\limits_{i=1}^n -y_{i} \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}
\\
\frac{1}{n} \sum\limits_{i=1}^n -y_{i} \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))} x_{i}  + \rho \omega
\end{array}\right]
$$
The second coordinate actually corresponds to p coordinates (there are the terms $x_{i}$ and $\rho \omega$) but I couldn't find any easy way to write it.

**The Hessian matrix, a square matrix of size (p+1) x (p+1). It is constituted of a 1x1 array (double derivative on $\omega_{0}$, in the top-left), a px1 and 1xp arrays (cross derivatives on $\omega$ and $\omega_{0}$, each one being the transposed of the other one, in the top-right and bottom-left), and a pxp array (double derivative on $\omega$, in the bottom-right). Here is the result :**

$$\nabla^{2} f_{1}: (\omega_0,\omega) \mapsto \begin{bmatrix}
\frac{1}{n} \sum\limits_{i=1}^n y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} & \frac{1}{n} \sum\limits_{i=1}^n y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i}^{T}\\
\frac{1}{n} \sum\limits_{i=1}^n y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i} & \frac{1}{n} \sum\limits_{i=1}^n y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i} x_{i}^{T} + \rho I_{pp}
\end{bmatrix}$$

By simply looking at how this Hessian matrix is constructed, we can determine right away it is symmetric. 
Now, for the convex proof, we will use a linear algebra argument. 

First, we decompose it as the sum of two different matrices, as follows :

$$\nabla^{2} f_{1} (\omega_0,\omega) = \begin{bmatrix} 
\frac{1}{n} \sum\limits_{i=1}^p y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} & \frac{1}{n} \sum\limits_{i=1}^p y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i}^{T}\\
\frac{1}{n} \sum\limits_{i=1}^p y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i} & \frac{1}{n} \sum\limits_{i=1}^p y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} x_{i} x_{i}^{T}
\end{bmatrix} + \begin{bmatrix} 0 & 0_{p}^{T}\\ 0_{p} & + \rho I_{pp} \end{bmatrix}$$

The main argument is the following : **the sum of two semi-definite positive matrices remains positive. Thus we will study both matrices.**    

For the left-hand one, that we will name A :
We can see that all columns are proportional to the first one, as we can simply multiply column $1$ by $x_{i}$ to get column $i+1$.
This means that this matrix is of rank 1. 
So all of its eigenvalues (that we will name $\lambda_{i}$) are $0$ except one. (to get into the details, we can write $dim(Ker(\nabla^{2} f_{1})) = p$ y seeing this matrix as a linear operator)   
We can determine the last one using the trace, as we know that the trace is equal to the sum of the eigenvalues, and as we can compute it explicitly as follows :   
$tr(A) = \sum\limits_{i=1}^{p+1} \lambda_{i} = \lambda = \frac{1}{n} \sum\limits_{i=1}^n y_{i}^2 \frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}} \ge 0$   
We finally get the last eigenvalue, positive as it is a sum of positive terms ($y_{i}^2$ and $\frac{\exp(-y_{i} (x_{i}^{T} \omega + \omega_{0}))}{(1 + \exp(-y_{i} (x_{i}^{T} \omega + \omega_{0})))^{2}}$ are of course positive). This proves that the first matrix is semi-definite positive.   

For the right-hand one, it is much easier : it is the identity matrix with a column / row of zeros, it is clearly diagonal so we can identify the eigenvalues very easily :  
$p$ eigenvalues are equal to $1$, and the last one is $0$. All are thus positive and the matrix is semi-definite positive.   

Which finally proves that the Hessian matrix of $f_{1}$ is symmetric, semi-definite positive, which means, $f_{1}$ is convex.  

Q3.2/ Code

In [35]:
#Fake & empty data to start with
n = np.size(y_test)
rho = 1/n

#A function computing the value of f1 :
def f1(w0,w):
    res = 0
    for i in range(n):
        res+=m.log(1+m.exp(-y_test[i]*(np.dot(eX_test[i],w)+w0)))
    return res/n + rho*np.dot(w,w)/2

def grad_f1(wo,w):
    tmp_down = np.ones((1,n))/n
    tmp_up = 0
    for i in range(n):
        expp = m.exp(-y[i]*(np.dot(xi,w)+w0))
        tmp_down += xi*(-y[i]/(1+expp)*(expp))
        tmp_up -= y[i]*expp
        #Pour que tout se passe bien, xi doit être un nparray. Idem w
    return np.concatenate((tmp_up, tmp_down), axis=0)

w_test = np.ones((n,1))
grad_f1(0,w_test)

In [34]:
#a = np.array([[1],[2]],[[1],[2]])
#b = np.array([[3],[4]],[[0],[0]])
p=3
c = np.ones((1,p))/p
c = c.reshape(p,1)
c

array([[0.33333333],
       [0.33333333],
       [0.33333333]])